In [ ]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

def build_model(input_shape):
    model = Sequential([
        Dense(64, activation='relu', input_shape=input_shape),
        Dense(32, activation='relu'),
        Dense(1, activation='linear')  # Single output for p-norm
    ])
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_absolute_error'])
    return model

def run_experiment_with_predefined_data_vectors(df):
    mse_per_example = []
    rmse_per_example = []
    mae_per_example = []

    df_experiment_results = pd.DataFrame(columns=['i', 'j', 'input_vectors', 'all_answers', 'actual_answers', 'predicted_answers', 'mse', 'rmse', 'mae'])

    unique_i_values = df['i'].unique()
    print("Unique 'i' values in dataframe:", unique_i_values)

    for i in unique_i_values:
        print(f"Processing for i={i} (number of prior examples)")
        experiments_for_i = df[df['i'] == i]

        mse_experiment = []
        mae_experiment = []
        rmse_experiment = []

        for _, row in experiments_for_i.iterrows():
            try:

                env = {"np": np, "array": np.array}
                vectors = eval(row['raw_questions'], env)

                answers = eval(row['raw_answers'], env)

                actual_p_norm = float(answers[-1])

            except (SyntaxError, ValueError, NameError, TypeError) as e:
                print(f"Error parsing row {row['i']}-{row['j']}: {e}")
                continue

            X = np.array([vector for vector in vectors])
            y = np.array(answers)

            # Use all but the last vector for training, and the last for testing
            X_train, X_test = X[:-1], X[-1].reshape(1, -1)
            y_train, y_test = y[:-1], np.array([actual_p_norm]).reshape(1, -1)

            model = build_model((X_train.shape[1],))
            model.fit(X_train, y_train, epochs=50, batch_size=10, verbose=0)

            y_pred = model.predict(X_test)
            mse = np.mean((y_test - y_pred) ** 2)
            mae = np.mean(np.abs(y_test - y_pred))
            rmse = np.sqrt(mse)

            mse_experiment.append(mse)
            mae_experiment.append(mae)
            rmse_experiment.append(rmse)

            temp_df = pd.DataFrame({
                'i': [row['i']],
                'j': [row['j']],
                'input_vectors': [vectors],  # Store all vectors used in this experiment
                'all_answers': [answers],  # Store all answers (training and test)
                'actual_answers': [actual_p_norm],  # The actual p-norm for the last vector
                'predicted_answers': [y_pred.flatten()[0]],
                'mse': [mse],
                'rmse': [rmse],
                'mae': [mae]
            })
            df_experiment_results = pd.concat([df_experiment_results, temp_df], ignore_index=True)

        mse_mean = np.mean(mse_experiment)
        mae_mean = np.mean(mae_experiment)
        rmse_mean = np.mean(rmse_experiment)

        mse_per_example.append(mse_mean)
        mae_per_example.append(mae_mean)
        rmse_per_example.append(rmse_mean)

        print(f"Vector Norm - Examples: {i}, MSE: {mse_mean}, MAE: {mae_mean}, RMSE: {rmse_mean}")

    df_average_results = pd.DataFrame({
        'i': unique_i_values,
        'average_mse': mse_per_example,
        'average_rmse': rmse_per_example,
        'average_mae': mae_per_example
    })

    return df_experiment_results, df_average_results

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

df = pd.read_csv('LLM_Final_results_P_Norm_2_experiment_results.csv')

df_experiment_results, df_average_results = run_experiment_with_predefined_data_vectors(df)

df_experiment_results_path = '/content/drive/My Drive/2NN_df_experiment_results_vector_2P_norm.csv'
df_average_results_path = '/content/drive/My Drive/2NN_df_average_results_vector_2P_norm.csv'

df_experiment_results.to_csv(df_experiment_results_path, index=False)
df_average_results.to_csv(df_average_results_path, index=False)

print(f'Results saved to: {df_experiment_results_path} and {df_average_results_path}')

Mounted at /content/drive
Unique 'i' values in dataframe: [  2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18  19
  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36  37
  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54  55
  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72  73
  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90  91
  92  93  94  95  96  97  98  99 100]
Processing for i=2 (number of prior examples)
1/1 [==============================] - 0s 78ms/step


<ipython-input-1-e547020ff45c>:85: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_experiment_results = pd.concat([df_experiment_results, temp_df], ignore_index=True)


1/1 [==============================] - 0s 52ms/step


1/1 [==============================] - 0s 53ms/step


Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 52ms/step
Vector Norm - Examples: 4, MSE: 5175.8708740609445, MAE: 63.67140412189413, RMSE: 63.67140412189413
Processing for i=5 (number of prior examples)
1/1 [==============================] - 0s 52ms/step
Vector Norm - Examples: 5, MSE: 2931.818364891461, MAE: 46.387561944855086, RMSE: 46.387561944855086
Processing for i=6 (number of prior examples)
1/1 [==============================] - 0s 56ms/step
Vector Norm - Examples: 6, MSE: 3445.4899368041, MAE: 45.89483518729028, RMSE: 45.89483518729028
Processing for i=7 (number of prior examples)
1/1 [==============================] - 0s 52ms/step
Vector Norm - Examples: 7, MSE: 2203.0478605314684, MAE: 36.145409179454255, RMSE: 36.145409179454255
Processing for i=8 (number of prior examples)
1/1 [==============================] - 0s 52ms/step
Vector Norm - Examples: 8, MSE: 1610.9561871344706, MAE: 30.89048133556089, RMSE: 30.89048133556089
Proce

In [ ]:
import numpy as np
import pandas as pd
from sklearn.linear_model import SGDRegressor
from sklearn.preprocessing import StandardScaler

def run_sgd_experiment_with_predefined_data_vectors(df):
    mse_per_example = []
    mae_per_example = []
    rmse_per_example = []

    df_experiment_results = pd.DataFrame(columns=['i', 'j', 'input_vectors', 'all_answers', 'actual_answers', 'predicted_answers', 'mse', 'rmse', 'mae'])

    unique_i_values = df['i'].unique()
    print("Unique 'i' values in dataframe:", unique_i_values)

    for i in unique_i_values:
        print(f"Processing for i={i} (number of prior examples)")
        experiments_for_i = df[df['i'] == i]

        mse_experiment = []
        mae_experiment = []
        rmse_experiment = []

        for _, row in experiments_for_i.iterrows():
            try:

                env = {"np": np, "array": np.array}
                vectors = eval(row['raw_questions'], env)
                answers = eval(row['raw_answers'], env)

                actual_p_norm = float(answers[-1])

            except (SyntaxError, ValueError, NameError, TypeError) as e:
                print(f"Error parsing row {row['i']}-{row['j']}: {e}")
                continue

            X = np.array(vectors)
            y = np.array(answers)

            X_train, X_test = X[:-1], X[-1].reshape(1, -1)
            y_train, y_test = y[:-1], np.array([actual_p_norm]).reshape(1, -1)

            scaler = StandardScaler()
            X_train = scaler.fit_transform(X_train)
            X_test = scaler.transform(X_test)

            sgd_regressor = SGDRegressor(max_iter=1000, tol=1e-3)
            sgd_regressor.fit(X_train, y_train)

            y_pred = sgd_regressor.predict(X_test)
            mse = np.mean((y_test - y_pred) ** 2)
            mae = np.mean(np.abs(y_test - y_pred))
            rmse = np.sqrt(mse)

            mse_experiment.append(mse)
            mae_experiment.append(mae)
            rmse_experiment.append(rmse)

            temp_df = pd.DataFrame({
                'i': [row['i']],
                'j': [row['j']],
                'input_vectors': [vectors],
                'all_answers': [answers],
                'actual_answers': [actual_p_norm],
                'predicted_answers': [y_pred.flatten()[0]],
                'mse': [mse],
                'rmse': [rmse],
                'mae': [mae]
            })
            df_experiment_results = pd.concat([df_experiment_results, temp_df], ignore_index=True)

        mse_mean = np.mean(mse_experiment)
        mae_mean = np.mean(mae_experiment)
        rmse_mean = np.mean(rmse_experiment)

        mse_per_example.append(mse_mean)
        mae_per_example.append(mae_mean)
        rmse_per_example.append(rmse_mean)

        print(f"SGD - Examples: {i}, MSE: {mse_mean}, MAE: {mae_mean}, RMSE: {rmse_mean}")

    df_average_results = pd.DataFrame({
        'i': unique_i_values,
        'average_mse': mse_per_example,
        'average_rmse': rmse_per_example,
        'average_mae': mae_per_example
    })

    return df_experiment_results, df_average_results


In [ ]:
from google.colab import drive

drive.mount('/content/drive')

df = pd.read_csv('LLM_Final_results_P_Norm_2_experiment_results.csv')

df_experiment_results, df_average_results = run_sgd_experiment_with_predefined_data_vectors(df)

df_experiment_results_path = '/content/drive/My Drive/SGD_df_experiment_results_vector_2_P_norm.csv'
df_average_results_path = '/content/drive/My Drive/SGD_df_average_results_vector_2_P_norm.csv'

df_experiment_results.to_csv(df_experiment_results_path, index=False)
df_average_results.to_csv(df_average_results_path, index=False)

print(f'Results saved to: {df_experiment_results_path} and {df_average_results_path}')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Unique 'i' values in dataframe: [  2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18  19
  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36  37
  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54  55
  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72  73
  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90  91
  92  93  94  95  96  97  98  99 100]
Processing for i=2 (number of prior examples)
SGD - Examples: 2, MSE: 1497.9927980483908, MAE: 30.405458037086536, RMSE: 30.405458037086536
Processing for i=3 (number of prior examples)


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1561: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
<ipython-input-3-24562a1c21a8>:78: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_experiment_results = pd.concat([df_experiment_results, temp_df], ignore_index=True)
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1561: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1561: ConvergenceWarning: Maxim

SGD - Examples: 3, MSE: 845366.0949748799, MAE: 221.98248554685833, RMSE: 221.98248554685833
Processing for i=4 (number of prior examples)
SGD - Examples: 4, MSE: 5131.957985866855, MAE: 45.72548773798574, RMSE: 45.72548773798574
Processing for i=5 (number of prior examples)


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1561: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1561: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1561: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1561: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stocha

SGD - Examples: 5, MSE: 3910.8338940481894, MAE: 48.04257417530824, RMSE: 48.04257417530824
Processing for i=6 (number of prior examples)
SGD - Examples: 6, MSE: 1844.7115477169539, MAE: 33.565558549082205, RMSE: 33.565558549082205
Processing for i=7 (number of prior examples)


...

SGD - Examples: 13, MSE: 1502.5472673822985, MAE: 32.11380341566886, RMSE: 32.11380341566886
Processing for i=14 (number of prior examples)
SGD - Examples: 14, MSE: 1236.5924941771204, MAE: 27.543367606465853, RMSE: 27.543367606465853
Processing for i=15 (number of prior examples)


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1561: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1561: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1561: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1561: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stocha

SGD - Examples: 15, MSE: 1311.6411762098974, MAE: 27.993543441947075, RMSE: 27.993543441947075
Processing for i=16 (number of prior examples)
SGD - Examples: 16, MSE: 1103.5209529781528, MAE: 25.469429950813687, RMSE: 25.469429950813687
Processing for i=17 (number of prior examples)
SGD - Examples: 17, MSE: 1434.0587944865883, MAE: 31.13167846454686, RMSE: 31.13167846454686
Processing for i=18 (number of prior examples)
SGD - Examples: 18, MSE: 1946.7662611054654, MAE: 33.68793655376395, RMSE: 33.68793655376395
Processing for i=19 (number of prior examples)


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1561: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1561: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1561: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1561: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(


SGD - Examples: 19, MSE: 951.6523129801695, MAE: 25.8049665715621, RMSE: 25.8049665715621
Processing for i=20 (number of prior examples)
SGD - Examples: 20, MSE: 904.8584658723935, MAE: 24.36905859164861, RMSE: 24.36905859164861
Processing for i=21 (number of prior examples)
SGD - Examples: 21, MSE: 793.8834184663591, MAE: 22.052045970716698, RMSE: 22.052045970716698
Processing for i=22 (number of prior examples)
SGD - Examples: 22, MSE: 1325.834351483532, MAE: 29.19058603096561, RMSE: 29.19058603096561
Processing for i=23 (number of prior examples)
SGD - Examples: 23, MSE: 987.3030051220062, MAE: 25.683283138000828, RMSE: 25.683283138000828
Processing for i=24 (number of prior examples)
SGD - Examples: 24, MSE: 846.9144981554628, MAE: 22.980240929611874, RMSE: 22.980240929611874
Processing for i=25 (number of prior examples)
SGD - Examples: 25, MSE: 1091.4654019793204, MAE: 26.25763658614002, RMSE: 26.25763658614002
Processing for i=26 (number of prior examples)
SGD - Examples: 26, MS